In [ ]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans

In [ ]:
import os

In [ ]:
print(os.listdir())

I- Data loading

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
submission = pd.read_csv('../input/sample_submission.csv')

**II -data exploration**

In [ ]:
train.head()

In [ ]:
test.head()

On va devoir modifier des parametre tel que la date pour pouvoir faire notre test

**III - data preprocessing**

Pour commencer on va regarder si il y a des valeurs manquantes

In [ ]:
plt.subplots(figsize=(18,6))
plt.title("outliers")
train.boxplot();


**III-Data visualisation¶**


In [ ]:
pass_count = train['passenger_count']
print("Maximum number of passengers on a trip : ", np.max(pass_count.values))
print("Minimum number of passengers on a trip : ", np.min(pass_count.values))
print("Average number of passengers on a trip : ", np.mean(pass_count.values))

f = plt.figure(figsize=(10,5))
pass_count = train['passenger_count'].value_counts()
sns.barplot(pass_count.index, pass_count.values, alpha=0.7)
plt.xlabel('Number of passengers on a trip', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.show()

grâce au deux graphique, on peut déduire que on va travailler sur ces deux éléments pour la trip distance et passenger

**IV-Features engineering¶**

On va changer les dates pour pouvoir faire notre test

In [ ]:
import datetime as dt

In [ ]:
train['pickup_datetime']= pd.to_datetime(train['pickup_datetime']) 

train['month'] = train['pickup_datetime'].dt.month

train['weekday'] = train['pickup_datetime'].dt.weekday

train['hours'] = train['pickup_datetime'].dt.hour

test['pickup_datetime']= pd.to_datetime(test['pickup_datetime']) 

test['month'] = test['pickup_datetime'].dt.month

test['weekday'] = test['pickup_datetime'].dt.weekday

test['hours'] = test['pickup_datetime'].dt.hour


Puis cadrer le test avec des formules simple

In [ ]:
train = train[(train.trip_duration < 5000)] 

In [ ]:
train = train[(train.pickup_longitude > -100)]
train = train[(train.pickup_latitude < 50)]


In [ ]:
train = train[(train.passenger_count > 0)]

In [ ]:
X_train = train.loc[:,['month','weekday','hours','passenger_count','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']]
X_test = test.loc[:,['month','weekday','hours','passenger_count','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']]
y_train = train['trip_duration'] 
X_train.shape, y_train.shape


In [ ]:
X_train.head()

Testons le model  avec xbgboost 

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

In [ ]:
xgb_model = xgb.XGBRegressor( max_depth=15, learning_rate=0.1, n_estimators=700, silent=True, objective='reg:linear', booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, importance_type='gain', )
xgb_model.fit(X_train, y_train)


In [ ]:
y_test_pred = xgb_model.predict(X_test)

In [ ]:
submission['trip_duration'] = y_test_pred
submission.head()

In [ ]:
submission.to_csv("submition.csv", index=False)